In [1]:
from bio_embeddings.embed import SeqVecEmbedder, ProtTransBertBFDEmbedder, ESM1bEmbedder, ProtTransT5BFDEmbedder, UniRepEmbedder, BeplerEmbedder, ESMEmbedder

# from bio_embeddings.project import tsne_reduce,umap_reduce
# from bio_embeddings.visualize import render_3D_scatter_plotly

from Bio import SeqIO
import pandas as pd 
import re 
import seaborn as sns
import os 


import numpy as np 

os.environ["CUDA_VISIBLE_DEVICES"]='1'

/xtdisk/xueyb_group/xutingfeng/Anaconda/envs/BioEmbedding/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [38]:
# raw = pd.read_csv("compatiable_incompatiable.csv")
# raw["SLF"] = raw["name"].apply(lambda x: x.split(":")[0])
# raw["RNase"] = raw["name"].apply(lambda x: x.split(":")[1])

# raw = raw[raw["RNase"] != "S19-RNase"]

# raw["SLF_seq"] = raw["SLF"].apply(lambda x: str(SeqIO.read(f"../data/fasta/single/{x}.fasta", "fasta").seq))

# raw["RNase_seq"] = raw["RNase"].apply(lambda x: str(SeqIO.read(f"../data/fasta/single/{x}.fasta", "fasta").seq))

# raw["label"] = raw["pair_type"].apply(lambda x: 0 if x=="incompatiable" else 1)

# # raw[["SLF", "RNase", "SLF_seq", "RNase_seq", "label"]].to_csv("dataset.csv", index=False)

对序列进行编码

In [ ]:
def embed_sis(sisFilePath, embedder):
    # if hasattr(embedder, "name"):
    #     embedderName = getattr(embedder, "name")
    # else:
    #     embedderName = 'unk'
    
    total = pd.read_csv(sisFilePath)
    for i in ["SLF_seq", "RNase_seq"]:
        embeddings = [embedder.reduce_per_protein(embedding) for embedding in embedder.embed_many(total[i])]
        # total[f"{i}_{embedderName}"] = pd.Series(embeddings)
        total[f"{i}_embedding"] = pd.Series(embeddings)

    return total 


In [ ]:
def fusion(*vec, methods="mean"):

    if methods == "mean":
        return np.mean(vec, axis=0)
    elif methods == "sum":
        return np.sum(vec, axis=0)
    elif methods == "concat":
        return np.concatenate(vec, axis=0)




In [22]:
embedder = ESMEmbedder()


In [41]:
filePath ="dataset.csv"

o = embed_sis(filePath, embedder)

In [42]:
import torch 

def ensure_dirs(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [44]:
import os.path as osp 
import torch 
outputDir = "../data/esmdata"
ensure_dirs(outputDir)
for idx, df in o.iterrows():

    fusion_name = df["SLF"] + "_" +df["RNase"]
    fusion_tensor = torch.tensor(fusion(df["SLF_seq_embedding"], df["RNase_seq_embedding"]))
    torch.save(fusion_tensor, osp.join(outputDir, f"{fusion_name}.pt"))
    




In [24]:
import h5py
import numpy as np 
import os.path as osp 

In [8]:
outputPath = "./test"
embedderList = [SeqVecEmbedder, ProtTransBertBFDEmbedder, ProtTransT5UniRef50Embedder, ProtTransXLNetUniRef100Embedder,ProtTransT5BFDEmbedder, ProtTransAlbertBFDEmbedder, ProtTransT5XLU50Embedder, BeplerEmbedder, ESM1bEmbedder, ESM1vEmbedder, ESMEmbedder,CPCProtEmbedder, PLUSRNNEmbedder]



for embedder in embedderList:
    embedderName = embedder.name if hasattr(embedder, "name") else "unk"
    o = embed_sis(filePath, embedder)

    with h5py.File(osp.join(outputPath, f"{embedderName}.hdf5"), "w") as f:
        data_group = f.create_group("test")

        data_group.create_dataset("SLF_seq_embedding", data=np.stack(o["SLF_Seq_embedding"].to_list(), axis=0).astype(np.float64))
        
        data_group.create_dataset("RNase_seq_embedding", data=np.stack(o["SRnase_Seq_embedding"].to_list(), axis=0).astype(np.float64))

,SLF,SLF_Seq,SRnase,SRnase_Seq,label,SLF_Seq_embedding,SRnase_Seq_embedding
0,S9-SLF1,MANGILKKLPEDLVFLILLTFPVKSLLRFKCISKAWSILIQSTTFI...,S11-RNase,YGAFDQIQLVLTWPPTYCHEKHCNRIPRNFTIHGLWPDNQHVMLNN...,1,"[-1.8425515, -0.039394815, 0.09533731, 1.27796...","[-1.0862033, -0.4538455, -0.44833943, -1.62088..."
1,S9-SLF1,MANGILKKLPEDLVFLILLTFPVKSLLRFKCISKAWSILIQSTTFI...,S7-RNase,YGVFDQIQLVLTWPPSFCHTKPCKRTPRNFTIHGLWPDDQHVLLND...,1,"[-1.8425515, -0.039394815, 0.09533731, 1.27796...","[-1.1834147, -0.4909198, -0.26537904, -1.49660..."
2,S9-SLF1,MANGILKKLPEDLVFLILLTFPVKSLLRFKCISKAWSILIQSTTFI...,S17-RNase,YGDFDYMQLVLTWPATFCYPKGFCQRIPPKNFTIHGLWPDKERQRL...,1,"[-1.8425515, -0.039394815, 0.09533731, 1.27796...","[-0.94042283, -0.764319, -0.28671283, -1.62076..."
3,S9-SLF1,MANGILKKLPEDLVFLILLTFPVKSLLRFKCISKAWSILIQSTTFI...,S1-RNase,YGNFEYLQLVLTWPASFCFRPKNICKRPAPKNFTIHGLWPEITGFR...,1,"[-1.8425515, -0.039394815, 0.09533731, 1.27796...","[-0.99790925, -0.47795296, -0.3325192, -1.5608..."
4,S9-SLF1,MANGILKKLPEDLVFLILLTFPVKSLLRFKCISKAWSILIQSTTFI...,S3-RNase,SANFDYFQLVLTWPASFCYPKNKCQRRSNNFTIHGLWPEKKRFRLE...,1,"[-1.8425515, -0.039394815, 0.09533731, 1.27796...","[-1.0127913, -0.36121148, -0.18399861, -1.6938..."
...,...,...,...,...,...,...,...
107,S10-SLF1,MANGILKKLPEDLVFLMLLTFPVKSLLRFKCISKAWSILIQSTSFI...,S17-RNase,YGDFDYMQLVLTWPATFCYPKGFCQRIPPKNFTIHGLWPDKERQRL...,1,"[-1.9062747, -0.045858134, 0.13149542, 1.19331...","[-0.94042283, -0.764319, -0.28671283, -1.62076..."
108,S10-SLF1,MANGILKKLPEDLVFLMLLTFPVKSLLRFKCISKAWSILIQSTSFI...,S1-RNase,YGNFEYLQLVLTWPASFCFRPKNICKRPAPKNFTIHGLWPEITGFR...,1,"[-1.9062747, -0.045858134, 0.13149542, 1.19331...","[-0.99790925, -0.47795296, -0.3325192, -1.5608..."
109,S10-SLF1,MANGILKKLPEDLVFLMLLTFPVKSLLRFKCISKAWSILIQSTSFI...,S3-RNase,SANFDYFQLVLTWPASFCYPKNKCQRRSNNFTIHGLWPEKKRFRLE...,1,"[-1.9062747, -0.045858134, 0.13149542, 1.19331...","[-1.0127913, -0.36121148, -0.18399861, -1.6938..."
110,S10-SLF1,MANGILKKLPEDLVFLMLLTFPVKSLLRFKCISKAWSILIQSTSFI...,S9-RNase,YGEFELLQLVLTWPASFCYANHCERIAPNNFTIHGLWPDNVTIRLQ...,1,"[-1.9062747, -0.045858134, 0.13149542, 1.19331...","[-1.0648934, -0.45515248, -0.25943485, -1.4592..."


In [25]:
o 

,SLF,SLF_Seq,SRnase,SRnase_Seq,label,SLF_Seq_embedding,SRnase_Seq_embedding
0,S9-SLF1,MANGILKKLPEDLVFLILLTFPVKSLLRFKCISKAWSILIQSTTFI...,S11-RNase,YGAFDQIQLVLTWPPTYCHEKHCNRIPRNFTIHGLWPDNQHVMLNN...,1,"[-1.8425515, -0.039394815, 0.09533731, 1.27796...","[-1.0862033, -0.4538455, -0.44833943, -1.62088..."
1,S9-SLF1,MANGILKKLPEDLVFLILLTFPVKSLLRFKCISKAWSILIQSTTFI...,S7-RNase,YGVFDQIQLVLTWPPSFCHTKPCKRTPRNFTIHGLWPDDQHVLLND...,1,"[-1.8425515, -0.039394815, 0.09533731, 1.27796...","[-1.1834147, -0.4909198, -0.26537904, -1.49660..."
2,S9-SLF1,MANGILKKLPEDLVFLILLTFPVKSLLRFKCISKAWSILIQSTTFI...,S17-RNase,YGDFDYMQLVLTWPATFCYPKGFCQRIPPKNFTIHGLWPDKERQRL...,1,"[-1.8425515, -0.039394815, 0.09533731, 1.27796...","[-0.94042283, -0.764319, -0.28671283, -1.62076..."
3,S9-SLF1,MANGILKKLPEDLVFLILLTFPVKSLLRFKCISKAWSILIQSTTFI...,S1-RNase,YGNFEYLQLVLTWPASFCFRPKNICKRPAPKNFTIHGLWPEITGFR...,1,"[-1.8425515, -0.039394815, 0.09533731, 1.27796...","[-0.99790925, -0.47795296, -0.3325192, -1.5608..."
4,S9-SLF1,MANGILKKLPEDLVFLILLTFPVKSLLRFKCISKAWSILIQSTTFI...,S3-RNase,SANFDYFQLVLTWPASFCYPKNKCQRRSNNFTIHGLWPEKKRFRLE...,1,"[-1.8425515, -0.039394815, 0.09533731, 1.27796...","[-1.0127913, -0.36121148, -0.18399861, -1.6938..."
...,...,...,...,...,...,...,...
107,S10-SLF1,MANGILKKLPEDLVFLMLLTFPVKSLLRFKCISKAWSILIQSTSFI...,S17-RNase,YGDFDYMQLVLTWPATFCYPKGFCQRIPPKNFTIHGLWPDKERQRL...,1,"[-1.9062747, -0.045858134, 0.13149542, 1.19331...","[-0.94042283, -0.764319, -0.28671283, -1.62076..."
108,S10-SLF1,MANGILKKLPEDLVFLMLLTFPVKSLLRFKCISKAWSILIQSTSFI...,S1-RNase,YGNFEYLQLVLTWPASFCFRPKNICKRPAPKNFTIHGLWPEITGFR...,1,"[-1.9062747, -0.045858134, 0.13149542, 1.19331...","[-0.99790925, -0.47795296, -0.3325192, -1.5608..."
109,S10-SLF1,MANGILKKLPEDLVFLMLLTFPVKSLLRFKCISKAWSILIQSTSFI...,S3-RNase,SANFDYFQLVLTWPASFCYPKNKCQRRSNNFTIHGLWPEKKRFRLE...,1,"[-1.9062747, -0.045858134, 0.13149542, 1.19331...","[-1.0127913, -0.36121148, -0.18399861, -1.6938..."
110,S10-SLF1,MANGILKKLPEDLVFLMLLTFPVKSLLRFKCISKAWSILIQSTSFI...,S9-RNase,YGEFELLQLVLTWPASFCYANHCERIAPNNFTIHGLWPDNVTIRLQ...,1,"[-1.9062747, -0.045858134, 0.13149542, 1.19331...","[-1.0648934, -0.45515248, -0.25943485, -1.4592..."


In [21]:



with h5py.File("h5py_example.hdf5", "wa") as f:
    data_group = f.create_group("test")
    data_group.create_dataset("SLF", data=np.stack(o["SLF"].to_list(), axis=0).astype(np.s))
    data_group.create_dataset("SLF_seq_embedding", data=np.stack(o["SLF_Seq_embedding"].to_list(), axis=0).astype(np.float64))
    
    data_group.create_dataset("RNase_seq_embedding", data=np.stack(o["SRnase_Seq_embedding"].to_list(), axis=0).astype(np.float64))



ValueError: Invalid mode; must be one of r, r+, w, w-, x, a

In [20]:
with h5py.File("h5py_example.hdf5", "r") as f:
    print(f["test"]["SLF_seq_embedding"])

<HDF5 dataset "SLF_seq_embedding": shape (112, 1280), type "<f8">


In [17]:
o["SRnase_Seq_embedding"].iloc[0].shape

(1280,)